## 다음 영화 리뷰 웹 스크래핑

In [7]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import psycopg2
import pandas as pd

conn_str = "host=localhost dbname=postgres user=postgres password=admin port=5432"
conn = psycopg2.connect(conn_str)
cur = conn.cursor()
cur.execute('''CREATE TABLE movie_review (number serial, user_id varchar(20), comment varchar(400), rating varchar(3), date varchar(20) )''') 
conn.commit()
number=120820 
base_url = "https://movie.daum.net/moviedb/grade?movieId={}"
driver = webdriver.Chrome("chromedriver.exe")

for x in range(113881, number+1):
    url=base_url.format(x)
    
    driver.get(url)
    driver.implicitly_wait(10)
    #elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "list_comment")))
    time.sleep(1)
    if len(driver.find_elements(By.CLASS_NAME,'alex_more')) !=0:
        set_click_root = driver.find_element(By.CLASS_NAME,'alex_more')
        set_item_click = set_click_root.find_elements(By.CLASS_NAME,'link_fold')
        #while len(set_item_click) != 0:
        set_item_click[0].click()
        time.sleep(2)
        #    set_item_click = set_click_root.find_elements(By.CLASS_NAME,'link_fold') 일단 제외
        #set_item_click[0].click()
        #print(len(set_item_click))
        #print("\n")
        #elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "list_comment")))
        elem = driver.find_element(By.CLASS_NAME, "list_comment")
        comment_item =elem.find_elements(By.TAG_NAME, "li")
        time.sleep(1)
        for i in range (len(comment_item)):
            rating=comment_item[i].find_element(By.CLASS_NAME,"ratings").text
            date=comment_item[i].find_element(By.CLASS_NAME,"txt_date").text
            user_id = comment_item[i].get_attribute("id")
            #print(comment_item[i].text)
            print(user_id)
            print(rating)
            print(date)
            if len(comment_item[i].find_elements(By.CLASS_NAME,"desc_txt")) != 0: #평점 달아놓고 논평 안한애들있음.
                comment=comment_item[i].find_element(By.CLASS_NAME,"desc_txt").text
                print(comment)
                cur.execute("INSERT INTO movie_review (user_id,comment,rating,date) VALUES (%s, %s,%s ,%s);",(user_id,comment,rating,date))
                conn.commit()
            if len(comment_item[i].find_elements(By.CLASS_NAME,"desc_txt")) == 0:
                cur.execute("INSERT INTO movie_review (user_id,rating,date) VALUES (%s,%s ,%s);",(user_id,rating,date))
                conn.commit()


<ipython-input-7-ae5037af5ce4>:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")


comment596772156
2
2017. 09. 03. 22:50
야망도 없고 대충 인생 즐기며 사는 주인공이 대기업 오너의 외동딸과 
결혼을 앞두고 누드 리플레이를 하는 과정을 통해 결혼의 진정한 의미를
깨닫고 성공적 결혼을 이룬 억지스런 설정의 코미디 인데..민망한 누드 반복도 
지루했고 뻔한 결말도 흥미롭지 못했다.
comment665496153
4
2021. 05. 16. 01:23
부족해
comment597493251
0
2019. 06. 18. 21:09
코미디라 기대가 컸다. 리플레이가 심해서 지루한감이... 안본 사람은 그냥 패스하길...
comment597396052
10
2019. 02. 19. 03:05
킬링타임으로 딱입니다 소재도 신선하고 재밋습니다!
comment597141246
6
2018. 05. 27. 05:51
그저 그랬다.
comment596917300
10
2017. 11. 26. 21:28
저도 잼있었어요
영화 평론가로서 10점 쌍따봉 드립니다. 
comment596844636
8
2017. 10. 13. 01:42
저는 재밌엇어용
comment596789359
10
2017. 09. 26. 00:01
재밌게봄
comment596781518
5
2017. 09. 15. 12:43
비슷한 영화가 너무 많음. 
comment597416943
10
2019. 03. 08. 01:17
빵빵터짐ㅋㅋㅋㅋ
comment598171272
3
2020. 08. 05. 13:31
comment597699933
7
2020. 05. 11. 02:26
편집등에서 조금씩 문제가 보이나 서양 영화치고는 귀신이 꽤 무섭게 잘 연출되었다
comment597697672
3
2020. 05. 05. 16:24
도시괴담 같은 이야기~ 에 시종 어두운 화면 은 긴장감보다는 짜증을 유발하는...
결말도 결말을위한 결말같은 급조...마지막까지 어두운 화면은 정말...
comment597532343
9
2019. 08. 11. 04:17
comment597367

KeyboardInterrupt: 

## 네이버 영화 리뷰 웹 스크래핑

In [9]:
import urllib.request
import bs4
import requests

def get_naver_movie_review(page_num, last_num):
    
    global dict
    dict = {'id' : [], 'star' : [], 'review' : []}
    

    for i in range(page_num, last_num+1):
        
        print("====={} 페이지=====".format(i))
        base_url = 'https://movie.naver.com/movie/point/af/list.naver?&page={}'
        url = base_url.format(i)
        resp = requests.get(url)

        soup =  bs4.BeautifulSoup(resp.text)
        tbody = soup.select_one('#old_content > table > tbody')
        trs = tbody.select('tr')

        for tr in trs : 
            tds = tr.select("td")
            
            id = tds[0].text.strip()
            print(id) # 리뷰 id
            dict['id'].append(id)

            star = tds[1].select_one("div") # 리뷰 별점
            star = star.text.strip().replace('별점 - 총 10점 중', "")
            print(star)
            dict['star'].append(star)

            review = tds[1].find("br").nextSibling # 리뷰 내용
            review = review.strip().replace('\n', "")
            print(review)
            dict['review'].append(review)

In [12]:
get_naver_movie_review(1, 10000)

=====1 페이지=====
17999562
10
난 재미있게봤는데 ㅋㅋㅋ 볼거리가 진짜 많음난 중국이 구한줄 알고 걱정했는데 중국어 3마디 끝이구만 뭐중국과 스토리가 관련도없고
17999561
2
그냥그랬다 액션은 괜찮았다
17999560
1
시작부터 참 거시기 허네~ 조선 시대 해적은 서양 해적들이 타는 배를 타고 다니는데...졸라 이질감 느껴짐...
17999559
10
그냥최고반전 굿굿꼭보세요
17999558
1
렉서스 광고로 1차 어질, 뜬금없는 쭝궈 여자 출현으로 2차 어질, 중국어 하는 흑인 남자 아이와 쭝궈 여자로 3차 어질, 스페이스 엑스 + 일런 머스크 찬양 4차 어질 개연성 1도 없는 전개 5차 어질... 보는 내내 달이 지구에 떨어졌으면 좋겠다라는 생각만 했습니다
17999557
8
시종일간 꽉막히고 답답한 분위기의 압박에 다이애나의 심정이 이해갔다
17999556
10
진짜 3편다 완벽한영화는 반지의제왕밖에 없는듯 합니다..진짜 안보신분들 꼭 보세요ㅠㅠ
17999555
10
잘 만들었네 불 끄고 보세요
17999554
10
썸머 워즈가 생각나네요. 하지만 너무 좋았습니다.
17999553
10
진짜 인생영화인듯 ㅠㅠㅠㅠ 짱입니다ㅠㅠ
=====2 페이지=====
17999552
3
너무나도 어설프고 유치한 디테일 수준이 놀라운 영화.
17999551
10
생산직 20년차 중소기업 현장관리직입니다.  사무직 일하는 친구들이 없어 몰랐는데 이영화 진짜 현실적인건가여 아님 현실인가여? 이게 지금 일어나는 일이라면 숨막혀 죽을거 같거나 그냥 돈 적게 받고 편한데 일하고 싶네여
17999550
10
진짜 이렇게 재밌는 영화 첨 봤어요 ㅠㅠㅠ인생영화♡♡
17999549
10
생각보다 재밌게 봤습니다
17999548
10

17999547
9
행복하게 해주는 명품 영화
17999546
9
일본영화  좋아함 잼남  좋아
17999545
10
정말 오랜만에 재밋었어요..! 코로나만 아니면 흥행할꺼같아요 중간중간 슬펏어영 ㅠㅠ
17999544
2
중간에 나

17999408
8
전쟁의 참혹함은 과거와 현재가 다르지 않을것이다..우크라이나의 평화를 기원합니다..^&^
17999407
10
이보다 더 완벽한 영화가 다시 나올수 있을까?
17999406
10
모두가 기다려온 그 배트맨이 왔다.
17999405
9
캡틴 아메리카… 역시 희생정신 최고!
17999404
1
중국, 렉서스 끝 서레기
17999403
10
ㄹㅇ 올해 가장 잘한 일ㅠㅠㅠ
17999402
10
영화보기10분전 평점이 왜이리 안좋아? 언니보단 재미있겠지~!
17999401
6
유하~~유튜브에서 저작권이고 뭐고 그냥 24시간 재생하면 좋을 영화
17999400
9
이 시대의 진짜 추리소설
17999399
10
아 너무 재밌는데 평점 왜이렇게 낮지? 존잼이다ㅜ우울했는데 정말 정말 마음이 따뜻해졌어요 제작진분들과 배우분들 감사합니다
=====17 페이지=====
17999398
10
생각과 사상의 차이를 국가가 통제하는 시대에 잔인한 고통을 이겨낸 작가 트럼보의 신념과 승리가 감동적입니다
17999397
1
이건 아니야… 무슨 특수부대가 방탄도 안쓰고 건물 수색할때도 머리먼저 들이밀고
17999396
10
OST가 최고입니다.
17999395
8
보는내내 크리스틴의 아름다움과 연기에 놀랐네요
17999394
8
노래가 좋습니다 메리미
17999393
10
정말 감동적인 수작입니다.
17999392
10
슬프지만 아름다운 영화입니다
17999391
8
흥미로웠습니다 오랜만에 우주영화를 보네요 근데 그래비티 등 다른 우주영화보다는 스케일이나 스토리가 조금 못미치네요
17999390
10
가만히 앉아있어도 귀여운 고양이들의 모습에 눈을 뗄 수 없었습니다. 고양이를 돌보는 사람도, 재개발로 삶의 터전을 옮겨야 했던 고양이들도 모두 행복했으면 좋겠습니다.
17999389
10
부족한게 없는..노래 추리재미 스토리 연애 연관성 성우 분들까지 전부다.. 제 인생 애니랍니다 다봐도 여러번씩보는데 안질린다죠 안보신분 꼭 봐보세요! 빠져서 해매실겁니다 강추!!
=====1

17999266
8
나는 과연 어떤 선택을 했을까...여자가 감당해야하는  현실...내딸에게 보여주기가 조심스럽다.교육효과 임신에 대해..낙태에 대해..너무 터부시 않는  전환이 필요하다.결혼전 간단하고 무해한 피임의 필요성이 많고  많은..생각의 마지막 끝이다.여자가 할 수 있는 무해한 피임의 허용...
17999265
9
할아버지 액션 영화..볼망하다
17999264
8
아무 정보없이 보기 시작했는데 끝엔 눈물이 났다 김남길 천우희 연기가 좋음
17999263
3
난 니 스펜서 ~ 스펜서 ~ 스펜서 ~
17999262
10
마음이 편안해지는 영화. .
17999261
10
중국영화라고 1점준애들... 이런 애들때문에 명작이 안알려지는거....
17999260
4
여자연기는 몰입을 깨네요 요즘 이렇게 연기 하는사람도 있나요 요즘 조연 들도 다들 잘하던데..
17999259
4
여자연기는 몰입을 깨네요 요즘 이렇게 연기 하는사람도 있나요 요즘 조연 들도 다들 잘하던데..
17999258
6
농인에 대한 연출은 클라이막스에 좋았지만 너무 뻔했음
17999257
1
착한 짱깨는 죽은 짱깨다 제발 헐리우드나 여타 다른 산업에 짱깨좀 그만 묻혀라 PC랑 페미똥 묻은거만 해도 개토나와
=====31 페이지=====
17999256
10
근 10년간 본 영화중 최고다
17999255
8
우리가 느끼는 지금의 고통과 불행은 어쩌면 아름다운 장미의 가시에 박힌거에 불과하다.
17999254
1
감상포인트를 꼭 체크해야 하는지 의문이네 [없음]도 추가해 주세요 이 영화는 왜 만들었는지 의문이 갈 정도니까
17999253
10
뭔 열 줄을 쓰래…굿 잘 봤어요
17999252
1
최민식씨 너무 실망입니다. 작금의 머한민국이 조선인민공화국보다 자유롭고 평화롭다고 할 수 있을까요? 진보적인 깨시민인줄 알았는데 북한 욕먹이는 영화에 출연하다니.. 최민식 너무 실망!!!!
17999251
10
웅장한 스케일과 예상치 못한 반전다른 사람 평점이 낮아서 걱정했는데 재밌거 보았습니다
1799

17999124
8
결코 상하지 않는 와인처럼 방황도 실패도 두려움도 묵묵히 담아낼 수 있기를
17999123
1
B급을 넘어서 c급 감성!!! 순간순간 2-30년전 심형래 영화 보는듯한 어이없고 실소터지게 하는 영화입니다. 관람객 132만명은 이용당했습니다.
17999122
10
풍경과 음식이 멋있습니다. 그리고 다이안레인의 아름다움은 정말 영화속의 풍경들과 잘 어울립니다.
17999121
2
스토리가 달과함께 떨어졌나봐요
17999120
6
여자 할머니 분장 너무 그지같아요 머리가발부터 분장이 전혀 할머니 분장 같지 않음. 잃어버린 아들은 진짜로 나이드신 할아버지 배우로 하셨는데 억지스러운 스토리 내용이 좀 많이 있는거 같음. 무당까지 불러놓고 저렇게 보내버리다니 어이없는 내용전개입니다. 어디 신XX 같은 사이비에서 지원받아 만든 영화인지 영화내용과 성당, 신부님은 전혀 없어도 될 내용 같은데 집어넣었어요. 마지막 실종된 아들이 좋아했던 여자친구는 뭔 깊은 사랑의 감정을 가졌길래 구슬로 목걸이도 만들고 눈물까지 즙을 짜는지 이해가 안감.
17999119
1
역시 손예진 역할이 컸다
17999118
10
헐..2005년도 작품임? 나 지금 보고 왔는데..ㄷㄷ / 연기 쩔고, 적절한  감동, 적절한 교훈..잘 만든 영화 / 야한? 영화 아님..그런 영화 찾으면 패스..
17999117
9
미국 하이틴 영화 중에 가장 공감되는 보편적인 내용이었다
17999116
10
최고입니다.  너무 감동이었어요
17999115
6
다큐와 영화를 넘나듬
=====45 페이지=====
17999114
8
영화관에서 볼만한 영화~ ㅎㅎ
17999113
1
어떻게 이런게 영화가 될 수 있나요? 투자받는게 이렇게 쉬운 걸까?
17999112
10
배우들이 다 오글거리지 않게 절제된 최고의 연기를 보여준 즐거운 영화였어요!!!
17999111
8
디워보다 100배 훌륭함.
17999110
2
;;; 무슨 영화징 보고 있음 섬마을 여선생 성폭행 사건이 생각 나긴하는데 그런걸 모티브는 아

17998991
10
이토록 감동적인 스파이더맨이라니
17998990
10

17998989
6
차승원,황정민 좋아해서 믿고 봤는데 백지는 연기도 그렇고 굳이 없어도 되는 듯 했고 배우한텐 죄송하지만 견자는 처음부터 끝까지 듣기 싫을 정도로 악만 쓰네요ㅠㅠ어제,오늘 두번 봤는데 황정민 연기와 차승원 치아만 기억이 남아요^^
17998988
8
이.수.경. 연기 굳 칭찬해
17998987
1
진짜 세상에서 가장 루즈했던영화감기는 눈 버텨가며 억지로봤는데개연성 스토리 연출 뭐하나 부족함없이 욕나옴
17998986
8
영상미와 연기가 인상적이었언 작품이었습니다
17998985
1
정말 수래기 같은 영화.. 파수꾼 감성을 노린것 같은데 현실은..
17998984
10
인생을 어떻게 살아야 하는지 차근차근 알려줬습니다.
17998983
1
화이트데이 맞죠? ㅋㅋ..
17998982
4
여자 간호사 예뻐서 끝까지 다볼수있었음
=====58 페이지=====
17998981
10
겨울을 생각했을 때 떠오르는 영화들은 많지만 겨울의 차가운 공기 냄새가 느껴지는 영화는 사바하가 유일할 것입니다. 참 처절하고 쓸쓸한 영화였어요.
17998980
10
잔잔한 감동영상미박정민의 연기내공+랩실력
17998979
7
전쟁에 대해 다시 한번 생각하게 되는 영화였다. 푸틴 썩을 놈...
17998978
10

17998977
3
매트릭스3, 터미네이터, 아마겟돈, 인디펜던스데이 다 섞은 중국 자본 잡탕영화. 진짜 중국 거는 걸러라
17998976
10
영상미도 은근있는 영화. 공감가고 엄마와딸이란 관계,아빠,친구,남친,여친 모든 그 관계들이 하나하나 소중할때여서 더욱 공감이 갔다이 지긋지긋한 곳을 떠나 사람많고,누구나 가고싶어하는곳으로 가서 살고싶지만 가끔은 또는 항상 기억나는 자라고 어쩔수없이 베어진 나의 고향도 현재진행형인 나인것을 인정하고 좋아하게되는
17998975
10
댓글들 가관이네 보자도 않은 놈들 ㅉㅉ SF 판타지들 좋아하시면 진심 볼만하다
17998974
8
킬링타임용으로 

...

17989741
6
연우진 때문에 봣는데ㅠ 여자배우가 연기를 너무못해요..  윽.. 그래서중간에나왓어요 ㅠㅋㅋ
17989740
10

17989739
10
픽사의 상상력은 그들을 찬양하게 한다
17989738
10
하아,,, 쿠키 영상 없구요ㅠ 크레딧 올라가자마자 물음표 세개 뜨고 굿바이 뜨는게 끝이라구요…. 바로 퇴장해주세요 쿠키없어요!!
17989737
10
너무재밌어요 꼭 보세여
17989736
4
개 촌구석에 테마파크 생기면 좋은거지 뭐 누가 악당이고 누가 선인지 모르것네. 그니까 2022년 대한민국땅에 복합쇼핑몰, 대형마트 하나 없는 곳이 있지. 딱 전라도 수준의 영화... 너무 유치하고 이상하게 느껴진다. 전직 깡패가 출마하며 대형테마파크 유치 반대한다는 설정도 너무 웃김. 영화 평 거의 써본 적 없는데 한 30분 째 보다가 같잖아서 평쓰러 옴. 진짜 북한주민들 이해 안되듯이 너무나 다른 사고방식과 상식에 놀라고 갑니다~
17989735
10
역시 믿고보는 최민식 배우님입니다^^ 오래간만의 파이도 루트도 반가웠습니다~~수학을 좋아했던 사람이라면 더 재밌을영화♡
17989734
6
많이 지루한 킹스맨
17989733
10
너므너무 재밌다.. ost도 딱
17989732
10
보러갈 사람들중에 mcu처럼 밝고 정의로운 히어로를 원하는 사람은 보지 마라. 어둡고 사람 잘 패는 히어로가 보고싶으면 추천
=====950 페이지=====
17989731
5
광명롯시] 왁자지껄 이것저것 차려놨는데 예식장 뷔페처럼 다 민숭민숭하다. '인디아나 존스'보다는 최근 한국 영화 '해적 2: 도깨비 깃발' 느낌이 났다. 별 캐릭터 없는 주연들의 재미없는 농담 들, 유치한 인위적 설정에 술술풀리며 피식~ 거리게 되는 극중 장면 등. 그냥 누워서 유튜브 보는 게 나았다고 생각한다.
17989730
10

17989729
10
재밌음 주연 배우들의 연기력 원툴 영화지만 그하나로 모든걸 커버한다
17989728
10
연출 연기 전부 뛰어난 최고의 영화!!
17989727
1
와 

17989571
6
ㅋㅋㄹㅇ개병맛이다..
17989570
10
시간 가는 줄 모르고 몰입하며 봤습니다 개인적으로 제스타일이라..굿굿
17989569
2
무슨 장르의 영화인지 모르겠응액션은 현실감도 타격감도 없고카체이싱 장면은 긴박감도 스케일도 없고어설픈 로맨스에 추리물인지 히어로물인지 쓸데없이 어마어마한 상영시간최악
17989568
10
지금까지와는 다른 배트맨
17989567
10
말이 필요 없다. N차 관람 마렵다. 남친이 같이 보러 안가주지만 오히려 좋다. 나의 고죠를 만나러 나는 혼자 영화관으로..이꾸요잇~!
=====966 페이지=====
17989566
1
킹갓 제네럴 전두환 장군을 기리는 영화
17989565
10
본질을 좀 때리는 느낌이었어요. 수학이란게 단순한 잣대로만 머물러서는 안되는데 말이죠
17989564
10

17989563
8
블란쳇의 통수 아프다
17989562
8
신혜선이 너무 이뻐서 영화 스토리에 집중이 안됨ㅇㅇ뭔 내용인지 모르고 봄
17989561
1
이재명 찢김 솨리 질럿!!!
17989560
9
최민식 배우님의 색다른 연기와, 가족끼리 볼 수 있을 정도의 잔잔하고 재미있는 내용 구성으로 만족합니다, 괴랄맞은 한국 영화만 나오더니 오랜만에 볼 만한게 나왔네요.
17989559
10
감동도 있고 믿고보는 최민식 연기에 몰입많이됨
17989558
10
어둡고 묵직한 영화 잘봤어요 런닝타임에 꽉차게 내용이 알차서 오히려 좋았어요
17989557
10
평점들 너무 냉혹하네...봤던 거라도 재밌게 행복하게 봐준 아이에게 감사하다.내 아이의 행복이 나의 돈보다 소중했다.
=====967 페이지=====
17989556
10
보면서 메디슨 카운티의 다리가 생각났습니다 연기가 너무 훌륭해서 보는 내내 안타까워서 눈물이 나올뻔 했어요.. 강추!
17989555
10
배우들의 연기도 좋았고, 이상과 현실을 한국 정치사의 한 단면으로 잘 표현한 영화.  조명이 인상적.
17989554
4
전 배트맨 시리즈랑 다르게 액션이나 cg 비중을줄이고 스

17989432
10
왕가위 감독님의 작품중 인생과 사랑에 대한 스토리를 잔잔하면서도 강렬하게 잘 풀어낸 작품.
17989431
6
최민식이 최민식한 영화   역시 배우 그 자체시네요
17989430
10
니콜라스 케이지의 연기력이 다시 한번 폭발한 작품.
17989429
6
무난한 한국영화에 수학을 얹다
17989428
6
부담없이 즐길수 있는 시리즈
17989426
10
마음에 닿는 음악이나 영화나 책, 사람 그 어느것도 없는 요즘이어서 너무 재미없던 삶이었는데 마음을 정면으로 건드리며 내가 나아갈 길이 무엇인지 어떻게 살고 싶은지 너무 분명한 목소리로 말해준 영화. 얼마만이야 눈물을 흘리며 한 장면도 놓치고 싶지 않던 영화가. 감사하다
17989425
8
로버트 패틴슨이 배트맨 역할을 맡았다고 했을 때는 기대를 별로 안했는데 상상 이상으로 연기를 너무 잘하고 어울려서 매우 놀랐으며 로버트 패틴슨을 다시 보게 되는 계기가 되었다.그리고 초반부가 지루하다는 의견이 많았는데 나는 오히려 매우 만족스러웠다, 매우 어두운 분위기 였는데 거기에 로버트 패틴슨의 목소리가 매우 잘 어울려서 좋았고 bgm도 분위기에 매우 잘 어울렸기에 더할 나위 없었다.하지만 러닝타임 세 시간은 확실히 부담스러운 정도였다.
17989424
10
마지막이 너무 슬픈영화
17989422
7
초반엔 좀 루즈한데 보면볼수록 몰입된다
17989421
5
화장실 장면에서 그래도 은근 기대했다.
=====980 페이지=====
17989420
10
이제껏 본 가장 인상적인 살인마 안톤 쉬거...
17989419
7
호리호리한 너드 작가 vs 신체 일부를 수집하는 덩치 큰 살인마. 여기서 오는 괴리감은 바톤 핑크가 자기 세계에 갇혀있는 것과 상응한다. 영화를 보는 내내 바톤 핑크의 자폐적인 시선을 따라가면서 독특한 경험을 하게 된다.
17989418
10
재미있게 보았어요
17989417
2
광수 컨셉 지겹다 웃기지도 않고
17989416
10
살아계셨으면…..그저 눈물만….
17989415
10
지

17989289
1
북한 사람은 잘 생기거나 혹은 똑똑하고 인간미 넘치는 사람으로만 연출하는 이유가 뭐냐? 지겹다 북한 미화
17989288
6
뮤지컬영화가 심하게 호불호 갈리는 입장에서 이건 그냥 괜찮다였다,,
17989287
8

17989285
1
진짜 오지호 어느 순간부터 작품 고르는 눈이 썩은 걸까 아니면 무슨 협박을 당해서 이딴 개노답 영화들만 작정하고 찍는 걸까? 아니 어떻게 찍는 영화마다 전부 스토리부터 연출까지 개노답의 향연이라 안깔수가 없네ㄹㅇㅋㅋ 이제 오지호 영화 개봉한다고 하면 믿고 거르는 배우로 생각해야겠음. 옛날에 환상의 커플, 내조의 여왕, 추노처럼 드라마만 찍던 시절엔 재밌는 작품들 많아서 좋아했었는데 정말이지 너무 실망스럽다.
=====993 페이지=====
17989284
10
왜 이걸 이제 봤지? 무려 13년전 영화인데 촌스럽지가 않네
17989283
7
스승의 은혜는 하늘 같아서 몸으로 갚겠다는데 뭐가 문제임?마침 스승도 참되거라 바르거라 몸으로 가르쳐주니 스승의 은혜는 우러러 볼수록 높아만지네
17989282
5
아니 .. 이게 흥미롭긴 함. 도대체 이게 무슨 내용일까 궁금해서 중간에 끊을 수가 없음. 근데 문제는 다 보고 나서도 무슨 내용인지 모르겠었음. 해석 보고 이해함… 내가 본 영화 중에 제일 이상한 영화임 .. 근데 재미 없지는 않음 엄청나게 무서운 장면은 없는데 그냥 영화 자체가 어둡고 소름돋음그리고 박해일 최고><
17989281
6
의식의 흐름대로 흘러가는 이야기.무슨 마르셸 프루스트의 잃어버린 시간을 찾아서도 아니고...
17989279
2
국산 망작 영화 대열에 오를 기세
17989278
4
지루+하품+졸음=단잠
17989277
10
평점 왜케 낮냐 이거 이 정도면 존나 웰메이든데
17989276
6
믿고보는 배우들이라 연기 연출 영상미 너무나 훌륭하고 좋았어요, 다만.. 시대적인 상황을 고려해서 봤지만서도.. 왜 아직도 영화계는 이렇게 한쪽으로만 치우 처져 있는 내용들을 담는건지 잘 모르겠군요
17989

AttributeError: 'NoneType' object has no attribute 'select'

### 데이터프레임 -> csv 저장

In [22]:
import pandas as pd

df = pd.DataFrame(dict)
df['star'] = pd.to_numeric(df['star'])
df

In [34]:
df_1 = df[df['star'] > 6]

df_1['label'] = 1
df_1

In [35]:
df_2 = df[df['star'] < 4]

df_2['label'] = 0
df_2

In [42]:
df = pd.concat([df_1, df_2])
df.drop(columns=['star'], inplace=True)
df.rename(columns={'review':'document'}, inplace=True)
df

,id,document,label
0,17999562,난 재미있게봤는데 ㅋㅋㅋ 볼거리가 진짜 많음난 중국이 구한줄 알고 걱정했는데 중국어...,1
3,17999559,그냥최고반전 굿굿꼭보세요,1
5,17999557,시종일간 꽉막히고 답답한 분위기의 압박에 다이애나의 심정이 이해갔다,1
6,17999556,진짜 3편다 완벽한영화는 반지의제왕밖에 없는듯 합니다..진짜 안보신분들 꼭 보세요ㅠㅠ,1
7,17999555,잘 만들었네 불 끄고 보세요,1
...,...,...,...
9969,17989233,만오천원에 세시간동안 고문당하고 싶다면 봐라.,0
9979,17989222,이보다 멍청할 순 없다. 남주. 고구마 좋아하시면 보시길,0
9984,17989217,진부한스토리에 고구같은 여주인공,0
9987,17989214,자다나옴 꿀잠잤다 ㅎㅎ,0


In [43]:
df.to_csv('./data/naver_train_new.csv', index=False)